#### Notebook with RAG-powered Q&A — respond to domain-specific questions using retrieved content

##### Change this field from markdown into code if you don't have the dependencies here.

import sys
!"{sys.executable}" -m pip install langchain langchain-community langchain-groq \
sentence-transformers chromadb > /dev/null 2>&1

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

C:\Users\Osazuwa-Ojo Victory\Documents\NYSC LAPO\Chatbot_Complete\comp-cb\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('Youghurt.txt', 'r') as f:
    text = f.read()

paragraphs = text.split("\n\n")  # split by double newlines
docs = [Document(page_content=para.strip()) for para in paragraphs if para.strip()]

In [3]:
# print(docs)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

split_docs = text_splitter.split_documents(docs)

In [5]:
# print(split_docs)

In [6]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

C:\Users\Osazuwa-Ojo Victory\AppData\Local\Temp\ipykernel_15524\2127729888.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [7]:
vectorstore = Chroma.from_documents(
    split_docs,
    embedding_model
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [10]:
import os
os.environ["GROQ_API_KEY"] = "Your Groq API Key"

In [11]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0
)

In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [13]:
query = "How long does youghurt preparation take?"
response = qa_chain(query)

print("Answer:")
print(response["result"])

print("\nSources:")
for doc in response["source_documents"]:
    print("-", doc.page_content)

C:\Users\Osazuwa-Ojo Victory\AppData\Local\Temp\ipykernel_15524\846852340.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain(query)


Answer:
Yogurt preparation involves two main steps: incubation and chilling/thickening. 

1. Incubation: 6 to 12 hours
2. Chilling: at least 4 hours
3. Optional thickening (for Greek-style yogurt): 2-4 hours

So, the total time can range from 12 hours (6 hours incubation + 4 hours chilling + 2 hours thickening) to 20 hours (12 hours incubation + 4 hours chilling + 4 hours thickening).

Sources:
- Chill for at least **4 hours** before eating â€“ this allows it to thicken further and develop flavor.
- Let the yogurt incubate undisturbed for **6 to 12 hours**. The longer it incubates, the tangier and thicker it becomes.
- For thicker, Greek-style yogurt:
1. After incubation, line a colander with cheesecloth or a clean kitchen towel
2. Place over a bowl and pour in the yogurt
3. Refrigerate and let drain for 2-4 hours (longer = thicker)
4. The liquid that drains off is whey â€“ save it for smoothies or baking!
